In [1]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import json

In [2]:
# soup = BeautifulSoup(html, "lxml")

In [3]:
# soup = BeautifulSoup(html, features="xml")

In [4]:

# # extract links from a page, credit: https://pythonspot.com/extract-links-from-webpage-beautifulsoup/
# # I had to <pip3 install lxml> to get this to work
# def getLinks(url):
#     html_page = urlopen(url)
#     soup = BeautifulSoup(html_page, features="lxml")
#     links = []

#     for link in soup.findAll('a', attrs={'href': re.compile("^/Movie Scripts/")}):
#         links.append(link.get('href'))

#     return links

# # this page is a list of all the movies IMSDb has on file for the Sci-Fi genre
# print( getLinks("https://imsdb.com/genre/Sci-Fi") )

# # Now, we need to wade through list of links generated by getLinks and extract
# # the urls for just the scripts, like is down below:

# # this is just the script of a movie
# URL = "https://imsdb.com/scripts/12-Monkeys.html"
# page = requests.get(URL)

# soup = BeautifulSoup(page.content, "html.parser")
# script = soup.find('pre')

# # print(script.text.strip())

### Uncomment the line below if you change the scraped genre in the scrape_scripts.py file

In [5]:
# !python scrape_scripts.py

In [6]:
#print(script.text.strip())

In [7]:
# blackpanther = "https://imsdb.com/scripts/Black-Panther.html"
# page_bp = requests.get(blackpanther)

# soup_bp = BeautifulSoup(page_bp.content, "html.parser")
# script_bp = soup_bp.find('pre')

In [8]:
# print(script_bp.text.strip())

In [9]:
# !pip install convokit

In [10]:
# from convokit import Corpus, download

In [11]:
# corpus = Corpus(filename=download("movie-corpus"))

In [12]:
from pandas.io.json import json_normalize

In [13]:
path_to_file = "scripts.txt"

In [14]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 6540867 characters


In [15]:
print(text[:250])

Quentin Tarantino's

                         R E S E R V O I R   D O G S








       October 22, 1990

                               -----------------


       This movie is dedicated to these following sources of

       inspi


In [16]:
# temp = open("movie-corpus/corpus.json")
# datum = json.load(temp)

In [17]:
# corpus = json_normalize(datum)

In [18]:
# corpus

In [19]:
# script_bp = script_bp.text.strip()

In [28]:
!pip install tensorflow

  Using cached tensorflow-2.6.2-cp36-cp36m-macosx_10_11_x86_64.whl (198.9 MB)


In [29]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [30]:
vocab = sorted(set(text.strip()))
print('{} unique characters'.format(len(vocab)))

96 unique characters


In [31]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [32]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

In [33]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[62, 63, 64, 65, 66, 67, 68], [85, 86, 87]]>

In [34]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [35]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [36]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [37]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [38]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(6540867,), dtype=int64, numpy=array([49, 82, 66, ..., 53, 50, 37])>

In [39]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [40]:
for ids in ids_dataset.take(13):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

Q
u
e
n
t
i
n
 
T
a
r
a
n


In [41]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [42]:
examples_per_epoch

64761

In [43]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'Q' b'u' b'e' b'n' b't' b'i' b'n' b' ' b'T' b'a' b'r' b'a' b'n' b't'
 b'i' b'n' b'o' b"'" b's' b'\r' b'\n' b'\r' b'\n' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b'R' b' ' b'E' b' ' b'S' b' ' b'E' b' '
 b'R' b' ' b'V' b' ' b'O' b' ' b'I' b' ' b'R' b' ' b' ' b' ' b'D' b' '
 b'O' b' ' b'G' b' ' b'S' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n'
 b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b'O'], shape=(101,), dtype=string)


In [44]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b"Quentin Tarantino's\r\n\r\n                         R E S E R V O I R   D O G S\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n       O"
b'ctober 22, 1990\r\n\r\n                               -----------------\r\n\r\n\r\n       This movie is dedicat'
b'ed to these following sources of\r\n\r\n       inspiration:\r\n\r\n\r\n                               TIMOTHY C'
b'AREY\r\n\r\n                                ROGER CORMAN\r\n\r\n                                ANDRE DeTOTH\r'
b'\n\r\n                                CHOW YUEN FAT\r\n\r\n                               JEAN LUC GODDARD\r\n'


In [45]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [46]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [47]:
dataset = sequences.map(split_input_target)

In [48]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"Quentin Tarantino's\r\n\r\n                         R E S E R V O I R   D O G S\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n       "
Target: b"uentin Tarantino's\r\n\r\n                         R E S E R V O I R   D O G S\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n       O"


In [49]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [50]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else: 
            return x

In [51]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [52]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [53]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 97) # (batch_size, sequence_length, vocab_size)


In [54]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  24832     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [55]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [56]:
sampled_indices

array([22,  1, 19, 32,  5, 74, 35, 74, 62, 86, 29,  9, 66, 48, 87, 90,  2,
       49, 70,  4,  2, 25, 60,  8, 86, 13, 48, 71, 95, 75,  9, 86, 79, 77,
       68, 85, 93, 33, 59,  9, 58, 96, 33, 72, 96, 57, 71, 53, 74, 65, 54,
       15, 88, 48, 34, 95, 92, 22, 39, 12, 86, 93, 83, 91, 69,  3, 47, 95,
        5, 79, 93, 24, 39,  7, 81, 41, 82, 19,  8, 59, 88, 82, 21, 32, 59,
       67,  2, 44, 41, 38, 79, 42, 88, 65, 35, 15, 58, 22, 84, 30])

In [57]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'             NICE GUY EDDIE\r\n                 I thought the cheatin wife shot\r\n                 Andy'

Next Char Predictions:
 b'4\t1?!mCmay;%ePz\xc2\x98\nQi \n7]$y)Pj\xc2\xbdn%yrpgx\xc2\xabA\\%Z\xc3\xa2Ak\xc3\xa2YjUmdV-}PB\xc2\xbd\xc2\x9d4G(y\xc2\xabv\xc2\x9ch\rO\xc2\xbd!r\xc2\xab6G#tIu1$\\}u3?\\f\nLIFrJ}dC-Z4w='


In [58]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [59]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.5757604


In [60]:
tf.exp(mean_loss).numpy()

97.101845

In [61]:
model.compile(optimizer='adam', loss=loss)

In [62]:
# Directory where the checkpoints will be saved
checkpoint_dir = './deneme2'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [67]:
EPOCHS = 20
# with 2 epochs, loss is 1.5116
# with 1 epoch, loss is 0.9197

In [68]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

1011/1011 [==============================] - 1575s 2s/step - loss: 0.9197


In [69]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature=temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = tf.constant([[0], [0]], dtype=tf.int64)
        tmp1 = tf.reshape(skip_ids, shape=(-1,))
        uniques, idx, counts = tf.unique_with_counts(tmp1)
        uniques_ids = tf.expand_dims(uniques, axis=1)
        
        sparse_mask = tf.SparseTensor(
    # Put a -inf at each bad index.
                values=[-float('inf')] * len(uniques_ids),
                indices=uniques_ids,
    # Match the shape to the vocabulary
                dense_shape=[len(ids_from_chars.get_vocabulary())])
        print(sparse_mask)
        
#         #skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
#         sparse_mask = tf.SparseTensor(
#         # Put a -inf at each bad index.
#             values=[-float('inf')]*len(skip_ids),
#             indices = skip_ids,
#         # Match the shape to the vocabulary
#             dense_shape=[len(ids_from_chars.get_vocabulary())]) 

        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
        predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    
    # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
        return predicted_chars, states

In [70]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

SparseTensor(indices=tf.Tensor([[0]], shape=(1, 1), dtype=int64), values=tf.Tensor([-inf], shape=(1,), dtype=float32), dense_shape=tf.Tensor([97], shape=(1,), dtype=int64))


In [74]:
start = time.time()
states = None
next_char = tf.constant(['N\'JOBU:'])
result = [next_char]

for n in range(10000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

N'JOBU:

D'ANGELO ANGLE

Hunsecker returns in the drink, squad cigarater, falling despertt between
Hunsecker off to Phosell, turning to Sidney.

                         HUNSECKER
                   (staring lawy)
            What are going to making they broud with
            ahand how come for?

                         SIDNEY
            Yesher!

                         SIDNEY
            Susie!...!

The steps in one of a squp what you
            will do is a smuld best good
            anything is waving me to you?

                         HUNSECKER
                   (eather)
            Why doubt you was from this, it
            favor.  Tearing it is a you!

He ties Steve looking at her, smiling at the back stole in
peramous struggles, unfulbed at me.J.J.  Gillis, comes in
          after her...NORMA REVERSE ANGLE

The waiter phone stares alone, heavily onto She that this movement
behind him.  They wite past young mood, in the abtob.
EVERY ACROSS, SOUNK coming toward Susan, 

In [75]:
print(type(result))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [97]:
text_script = str(result.numpy()[0].decode("utf-8"))
text_script

"N'JOBU:\r\n\r\nD'ANGELO ANGLE\r\n\r\nHunsecker returns in the drink, squad cigarater, falling despertt between\r\nHunsecker off to Phosell, turning to Sidney.\r\n\r\n                         HUNSECKER\r\n                   (staring lawy)\r\n            What are going to making they broud with\r\n            ahand how come for?\r\n\r\n                         SIDNEY\r\n            Yesher!\r\n\r\n                         SIDNEY\r\n            Susie!...!\r\n\r\nThe steps in one of a squp what you\r\n            will do is a smuld best good\r\n            anything is waving me to you?\r\n\r\n                         HUNSECKER\r\n                   (eather)\r\n            Why doubt you was from this, it\r\n            favor.  Tearing it is a you!\r\n\r\nHe ties Steve looking at her, smiling at the back stole in\r\nperamous struggles, unfulbed at me.J.J.  Gillis, comes in\r\n          after her...NORMA REVERSE ANGLE\r\n\r\nThe waiter phone stares alone, heavily onto She that this movement\r

In [128]:
import string

In [144]:
### Model Evaluation: Count # of Nonsense Words
new_script = text_script.strip()
new_script = new_script.translate(str.maketrans('', '', string.punctuation))

new_script = new_script.replace('\r', ' ').replace('\n', ' ').replace('   ', '').replace('  ', ' ')
new_script

'NJOBU DANGELO ANGLE Hunsecker returns in the drink squad cigarater falling despertt between Hunsecker off to Phosell turning to Sidney HUNSECKERstaring lawy What are going to making they broud with ahand how come for SIDNEY Yesher SIDNEY Susie The steps in one of a squp what you will do is a smuld best good anything is waving me to you HUNSECKEReather Why doubt you was from this it favor Tearing it is a you He ties Steve looking at her smiling at the back stole in peramous struggles unfulbed at meJJ Gillis comes inafter herNORMA REVERSE ANGLE The waiter phone stares alone heavily onto She that this movement behind him They wite past young mood in the abtob EVERY ACROSS SOUNK coming toward Susan hand into the club HUNSECKER You want todayvaribly Because I levew Hunsecretting smile activates his face trained up her head on the bed and takes across the outain of this arm tolights the reaction to jump Hunsecker is amaded upwards HUNSECKERexaminion Betty JJ Im jot looking to peramous wine 

In [145]:
new_script_words = new_script.split(' ')
new_script_words

['NJOBU',
 'DANGELO',
 'ANGLE',
 'Hunsecker',
 'returns',
 'in',
 'the',
 'drink',
 'squad',
 'cigarater',
 'falling',
 'despertt',
 'between',
 'Hunsecker',
 'off',
 'to',
 'Phosell',
 'turning',
 'to',
 'Sidney',
 'HUNSECKERstaring',
 'lawy',
 'What',
 'are',
 'going',
 'to',
 'making',
 'they',
 'broud',
 'with',
 'ahand',
 'how',
 'come',
 'for',
 'SIDNEY',
 'Yesher',
 'SIDNEY',
 'Susie',
 'The',
 'steps',
 'in',
 'one',
 'of',
 'a',
 'squp',
 'what',
 'you',
 'will',
 'do',
 'is',
 'a',
 'smuld',
 'best',
 'good',
 'anything',
 'is',
 'waving',
 'me',
 'to',
 'you',
 'HUNSECKEReather',
 'Why',
 'doubt',
 'you',
 'was',
 'from',
 'this',
 'it',
 'favor',
 'Tearing',
 'it',
 'is',
 'a',
 'you',
 'He',
 'ties',
 'Steve',
 'looking',
 'at',
 'her',
 'smiling',
 'at',
 'the',
 'back',
 'stole',
 'in',
 'peramous',
 'struggles',
 'unfulbed',
 'at',
 'meJJ',
 'Gillis',
 'comes',
 'inafter',
 'herNORMA',
 'REVERSE',
 'ANGLE',
 'The',
 'waiter',
 'phone',
 'stares',
 'alone',
 'heavily',
 

In [150]:
#!pip install pyenchant

  Using cached pyenchant-3.2.2-py3-none-any.whl (55 kB)


In [147]:
#!brew update
#!brew install enchant

In [153]:
import enchant
d = enchant.Dict("en_US")
script_english_words = [d.check(i) for i in new_script_words]

In [154]:
pct_english_words = sum(script_english_words)/len(script_english_words)
pct_english_words

0.7885245901639344

next steps: find nonsense words, count num nonsense/normal words --> success metric

Code used: [the original script](https://github.com/tarik3333/Movie-script-gen/blob/main/Untitled9.ipynb) and [this answer](https://stackoverflow.com/questions/68939916/tensorflow-text-generation-rnn-example-failing-on-tf-2-6-tf-sparse-to-dense)